In [1]:
import zipfile
import glob
import os

In [2]:
working_dir = os.getcwd()
print(f"You are now working in {working_dir}")
print("If you need to change to the parent directory, run the cell below")

You are now working in c:\Users\fredd\Desktop\Studier\Project5\flowers\flowers_CNN\jupyter_notebooks
If you need to change to the parent directory, run the cell below


In [3]:
os.chdir(os.path.dirname(working_dir))

In [4]:
working_dir = os.getcwd()
print(f"You have now changed your working directory to {working_dir}")

You have now changed your working directory to c:\Users\fredd\Desktop\Studier\Project5\flowers\flowers_CNN


In [5]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()

In [6]:
KaggleDatasetPath = "kurito/flower-photos"
DestinationFolder = "inputs/dataset/raw"
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

Dataset URL: https://www.kaggle.com/datasets/kurito/flower-photos
License(s): unknown




  0%|          | 0.00/1.48G [00:00<?, ?B/s]
  0%|          | 1.00M/1.48G [00:00<17:02, 1.55MB/s]
  0%|          | 3.00M/1.48G [00:00<05:54, 4.47MB/s]
  0%|          | 6.00M/1.48G [00:00<02:48, 9.37MB/s]
  1%|          | 9.00M/1.48G [00:01<01:58, 13.4MB/s]
  1%|          | 12.0M/1.48G [00:01<01:34, 16.7MB/s]
  1%|          | 15.0M/1.48G [00:01<01:27, 18.0MB/s]
  1%|          | 18.0M/1.48G [00:01<01:39, 15.8MB/s]
  1%|▏         | 20.0M/1.48G [00:01<01:42, 15.3MB/s]
  1%|▏         | 22.0M/1.48G [00:01<01:58, 13.2MB/s]
  2%|▏         | 25.0M/1.48G [00:02<01:35, 16.3MB/s]
  2%|▏         | 28.0M/1.48G [00:02<01:23, 18.7MB/s]
  2%|▏         | 32.0M/1.48G [00:02<01:05, 23.6MB/s]
  2%|▏         | 35.0M/1.48G [00:02<01:05, 23.8MB/s]
  3%|▎         | 39.0M/1.48G [00:02<00:56, 27.5MB/s]
  3%|▎         | 42.0M/1.48G [00:02<01:06, 23.3MB/s]
  3%|▎         | 45.0M/1.48G [00:02<01:12, 21.2MB/s]
  3%|▎         | 49.0M/1.48G [00:03<01:01, 25.0MB/s]
  3%|▎         | 52.0M/1.48G [00:03<01:13, 20.8MB/s]
 

In [ ]:
DestinationFolder = "inputs/dataset/raw"
KaggleJsonPath = "kaggle.json"

# This will find the zip-file in the DestinationFolder and unzip
for zip_path in glob.glob(f"{DestinationFolder}/*.zip"):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(DestinationFolder)
    os.remove(zip_path)  # Removes zip-file after unpacking

if os.path.exists(KaggleJsonPath):
    os.remove(KaggleJsonPath) # This will remove the kaggle.json file when unzipping the files are done